In [44]:
from picamera.array import PiRGBArray
from picamera import PiCamera
import time
import cv2
import numpy as np
from fractions import Fraction

camera = PiCamera()
#res = (1640, 1232)
res = (640, 480)
#res = (1280, 960)
camera.resolution = res
#camera.framerate = 30

time.sleep(1)
camera.iso = 100
t = camera.awb_gains
print("t=", t)
camera.awb_mode = 'off'
camera.awb_gains = t
#camera.digital_gain = Fraction(1, 1)
camera.drc_strength = 'off'
camera.exposure_mode = 'off'
rawCapture = PiRGBArray(camera, size=res)

time.sleep(0.1)
n = 0
t1 = time.time()
averf = 10
nmask = 0
nnmask = 0
xcen = 280
ycen = 210

mdc = np.ones(mask.shape, np.uint8)*255
mask_diff_color = cv2.circle(mdc,(xcen,ycen), 190, 0, -1)

for frame in camera.capture_continuous(rawCapture, format="bgr", use_video_port=True):
    
    img = frame.array
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    low1 = np.array([0, 50, 60])
    low2 = np.array([170, 50, 60])
    low3 = np.array([0, 50, 60])
    high1 = np.array([20, 255, 255])
    high2 = np.array([180, 255, 255])
    high3 = np.array([30, 160, 255])
    mask1 = cv2.inRange(hsv, low1, high1)
    mask2 = cv2.inRange(hsv, low2, high2)
    mask = cv2.bitwise_or(mask1, mask2)
    mask_dc = cv2.inRange(hsv, low3, high3)
    mask_dc = cv2.bitwise_and(mask_diff_color, mask_dc)
    mask11 = cv2.bitwise_or(mask, mask_dc)

    if n==0:
        nmask = mask
    
    if n < 30:
        nmask, nnmask = create_not_mask(nmask, mask)
    #image = cv2.imdecode(image, 1)
    #print(image.shape)
    #cv2.imshow("Frame", img)
    #cv2.imshow("Frame2", mask)
    #print(camera.iso, camera.digital_gain, camera.analog_gain)
    mask = cv2.erode(mask, np.ones((3, 3))/9)
    mask = cv2.dilate(mask, np.ones((3, 3))/9)
    
    key = cv2.waitKey(1) & 0xFF
    
    print(1/(time.time()-t1), averf, end = '\r')
    averf = averf*0.9+0.1*1/(time.time()-t1)
    #print("average = ", averf)
    t1 = time.time()
    rawCapture.truncate(0)
    cv2.imshow('masks', img)#cv2.resize(img, (300, 200)))
    cv2.imshow('mask', mask)#cv2.resize(mask, (300, 200)))
    cv2.imshow('not mask', nnmask)
    cv2.imshow('cleaned', cv2.bitwise_and(mask, nnmask))
    cv2.imshow('imn', cv2.bitwise_and(img,img,mask = mask_diff_color))
    cv2.imshow('mdc', mask_dc)
    if key == ord("q") or n>5000:
        cv2.imwrite('nnmask.png', nnmask)
        #cv2.imwrite('mask_diff_color.png', mask_diff_color)
        break
    n += 1
    

cv2.destroyAllWindows()
camera.close()

t= (Fraction(455, 256), Fraction(435, 256))


In [2]:
def create_not_mask(mask, noise):
    mask = cv2.bitwise_or(mask, noise)
    nnmask = cv2.bitwise_not(nmask)
    return mask, nnmask

In [25]:

cv2.destroyAllWindows()
camera.close()

In [23]:
help(cv2.inRange)

Help on built-in function inRange:

inRange(...)
    inRange(src, lowerb, upperb[, dst]) -> dst
    .   @brief  Checks if array elements lie between the elements of two other arrays.
    .   
    .   The function checks the range as follows:
    .   -   For every element of a single-channel input array:
    .   \f[\texttt{dst} (I)= \texttt{lowerb} (I)_0  \leq \texttt{src} (I)_0 \leq  \texttt{upperb} (I)_0\f]
    .   -   For two-channel arrays:
    .   \f[\texttt{dst} (I)= \texttt{lowerb} (I)_0  \leq \texttt{src} (I)_0 \leq  \texttt{upperb} (I)_0  \land \texttt{lowerb} (I)_1  \leq \texttt{src} (I)_1 \leq  \texttt{upperb} (I)_1\f]
    .   -   and so forth.
    .   
    .   That is, dst (I) is set to 255 (all 1 -bits) if src (I) is within the
    .   specified 1D, 2D, 3D, ... box and 0 otherwise.
    .   
    .   When the lower and/or upper boundary parameters are scalars, the indexes
    .   (I) at lowerb and upperb in the above formulas should be omitted.
    .   @param src first input 